In [0]:
from pyspark.sql.functions import sum as _sum, to_date

# Read Silver sales
silver_sales_df = spark.read.table("workspace.default.silver_sales_transactions")

# Aggregate daily sales
gold_daily_sales_df = silver_sales_df.groupBy(to_date("transaction_date").alias("sales_date")) \
    .agg(
        _sum("total_amount").alias("total_daily_revenue"),
        _sum("quantity").alias("total_quantity_sold")
    )

# Write Gold table
gold_daily_sales_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.default.gold_daily_sales")


In [0]:
from pyspark.sql.functions import month, year

# Read Silver tables
silver_store_df = spark.read.table("workspace.default.silver_store_region")

# Join sales with store region
sales_region_df = silver_sales_df.join(
    silver_store_df,
    on="store_id",
    how="left"
)

# Aggregate monthly revenue by region
gold_monthly_region_df = sales_region_df.groupBy(
    year("transaction_date").alias("year"),
    month("transaction_date").alias("month"),
    "region",      
    "country"       
).agg(
    _sum("total_amount").alias("monthly_revenue"),
    _sum("quantity").alias("monthly_quantity_sold")
)

# Write Gold table
gold_monthly_region_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.default.gold_monthly_revenue_by_region")


In [0]:
# Read Product Master
silver_product_df = spark.read.table("workspace.default.silver_product_master")

# Join sales with product details
sales_product_df = silver_sales_df.join(
    silver_product_df,
    on="product_id",
    how="left"
)

# Aggregate product metrics
gold_product_metrics_df = sales_product_df.groupBy(
    "product_id",
    "product_name",     
    "category"          
).agg(
    _sum("total_amount").alias("total_revenue"),
    _sum("quantity").alias("total_quantity_sold"),
    (_sum("total_amount") / _sum("quantity")).alias("avg_unit_price")
)

# Write Gold table
gold_product_metrics_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.default.gold_product_metrics")


In [0]:
%sql
-- Optimize daily sales
OPTIMIZE workspace.default.gold_daily_sales ZORDER BY (sales_date);

-- Optimize monthly revenue by region
OPTIMIZE workspace.default.gold_monthly_revenue_by_region ZORDER BY (country, region);

-- Optimize product metrics
OPTIMIZE workspace.default.gold_product_metrics ZORDER BY (category, product_id);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 2004), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1766449722064, 1766449722551, 8, 0, null, List(0, 0), null, 6, 6, 0, 0, null)"
